Parámetros:

Habitación de MxN espacios
Número de corredores
Número máximo de steps (para ejecutar el modelo)
Realiza la siguiente simulación:

Inicializa posiciones aleatorias para los corredores
En cada step:
- Si aún no tocan la primer esquina
Los corredores se mueven una celda en dirección de su esquina más cercana.
- Si ya tocaron su primer esquina
- Los corredores ahora se mueven dos celdas (en cada step) en dirección de su segunda esquina inicial más cercana*. (Se mueven dos celdas siempre y cuando sea posible)
Si tocan la segunda esquina
Los corredores dejan de moverse.
La simulación termina una vez que se haya alcanzado el número máximo de steps.
Con ayuda de matplotlib debes graficar la simulación (como hicimos en actividades previas)
En una celda de texto o en un comentario, escribe tus conclusiones de esta actividad.

In [1]:
#%pip install mesa
from mesa import Agent, Model

from mesa.space import MultiGrid

from mesa.time import SimultaneousActivation

from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd

import time
import datetime
import random

In [2]:
from random import randint

In [12]:
from typing import Any


def get_grid(model: "Habitacion") -> np.ndarray:
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
      cell_content, (x, y) = cell
      for obj in cell_content:
        if isinstance(obj, Corredor):
          grid[x][y] = 2
        elif isinstance(obj, Celda):
          grid[x][y] = obj.estado # 0 o 1
    return grid

class Corredor(Agent):
  model: "Habitacion"
  
  def __init__(self, unique_id, model, posicion: tuple[int, int]):
    super().__init__(unique_id, model)
    self.posicion = posicion
    self.esq1, self.esq2 = self.get_esquinas()
    self.primerEsquina = False

  def step(self):
    #print(f"corredor {self.unique_id}")
    print(f"pos {self.pos}")
    newX = self.pos[0]
    newY = self.pos[1]
    if not self.primerEsquina:
      # eje x
      if self.pos[0] < self.esq1[0]:
        newX = self.pos[0] + 1
      elif self.pos[0] > self.esq1[0]:
        newX = self.pos[0] - 1
      else:
        pass

      # eje y
      if self.pos[1] < self.esq1[1]:
        newY = self.pos[1] + 1
      elif self.pos[1] > self.esq1[1]:
        newY = self.pos[1] - 1
      else:
        pass

      if self.pos == self.esq1:
        self.primerEsquina = True

    else:
      # eje x
      if self.pos[0] < self.esq2[0]:
        newX = self.pos[0] + 2
      elif self.pos[0] > self.esq2[0]:
        newX = self.pos[0] - 2
      else:
        pass

      # eje y
      if self.pos[1] < self.esq2[1]:
        newY = self.pos[1] + 2
      elif self.pos[1] > self.esq2[1]:
        newY = self.pos[1] - 2
      else:
        pass

      if newX < 0:
        newX = 0
      elif newX > self.model.grid.width-1:
        newX = self.model.grid.width-1
      if newY < 0:
        newY = 0
      elif newY > self.model.grid.height-1:
        newY = self.model.grid.height-1

    self.model.grid.move_agent(self, (newX, newY))

  def get_esquinas(self):
    esquinas = [(0, 0),
                (self.model.grid.width-1, 0),
                (0, self.model.grid.height-1),
                (self.model.grid.width-1, self.model.grid.height-1)
    ]
    # ordenar esquinas por distancia a la posicion del agente
    esquinas.sort(key=lambda corner: max(abs(self.posicion[0]-corner[0]), abs(self.posicion[1]-corner[1])))
    return esquinas[0], esquinas[1]

class Celda(Agent):
  def __init__(self, unique_id, model, _estado):
    super().__init__(unique_id, model)

  def step(self):
    pass

class Habitacion(Model):
  def __init__(self, M, N, num_corredores, *args: Any, **kwargs: Any):
    super().__init__(*args, **kwargs)
    self.grid = MultiGrid(M, N, False)
    self.schedule = SimultaneousActivation(self)
    self.num_corredores = num_corredores

    # creacion de agentes
    for i in range(self.num_corredores):
      # conseguir coordenadas aleatorias
      x = randint(0, M-1)
      y = randint(0, N-1)
      agente = Corredor(f"corredor {i}", self, (x,y))
      print(f"x {x}, y {y}")
      self.grid.place_agent(agente, (x,y))
      self.schedule.add(agente)

    self.datacollector = DataCollector(
        model_reporters={"Grid": get_grid}
        )

  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()


In [13]:
M = 15
N = 10
num_corredores = 1

iteraciones = 20

model = Habitacion(M, N, num_corredores)

for i in range(iteraciones):
  model.step()


x 5, y 6
type(model)=<class '__main__.Habitacion'>
pos (5, 6)
type(model)=<class '__main__.Habitacion'>
pos (4, 7)
type(model)=<class '__main__.Habitacion'>
pos (3, 8)
type(model)=<class '__main__.Habitacion'>
pos (2, 9)
type(model)=<class '__main__.Habitacion'>
pos (1, 9)
type(model)=<class '__main__.Habitacion'>
pos (0, 9)
type(model)=<class '__main__.Habitacion'>
pos (0, 9)
type(model)=<class '__main__.Habitacion'>
pos (0, 7)
type(model)=<class '__main__.Habitacion'>
pos (0, 5)
type(model)=<class '__main__.Habitacion'>
pos (0, 3)
type(model)=<class '__main__.Habitacion'>
pos (0, 1)
type(model)=<class '__main__.Habitacion'>
pos (0, 0)
type(model)=<class '__main__.Habitacion'>
pos (0, 0)
type(model)=<class '__main__.Habitacion'>
pos (0, 0)
type(model)=<class '__main__.Habitacion'>
pos (0, 0)
type(model)=<class '__main__.Habitacion'>
pos (0, 0)
type(model)=<class '__main__.Habitacion'>
pos (0, 0)
type(model)=<class '__main__.Habitacion'>
pos (0, 0)
type(model)=<class '__main__.Habitaci

In [5]:
all_grid = model.datacollector.get_model_vars_dataframe()
# print(all_grid.to_string())

In [7]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [8]:
anim

/var/folders/vn/j0cd38593bq8df85jk62mb0r0000gn/T/ipykernel_13950/1752693568.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  patch.set_data(all_grid.iloc[i][0])
